In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
dataset=pd.read_csv('../data/diabetes.csv')
print(dataset.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [3]:
zero_not_accepted=['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for column in zero_not_accepted:
    dataset[column]=dataset[column].replace(0,np.NaN)
    mean=int(dataset[column].mean(skipna=True))
    dataset[column]=dataset[column].replace(np.NaN,mean) 

In [4]:
X=dataset.iloc[:,0:8]
y=dataset.iloc[:,8]
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)
X_sc=StandardScaler()
X_train=X_sc.fit_transform(X_train)
X_test=X_sc.transform(X_test)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


In [5]:
###SKlearn KNN 
clf=KNeighborsClassifier(n_neighbors=11,p=2,metric='euclidean')
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
print("confusion matrix:\n",cm)
f1=f1_score(y_test,y_pred)
print("f1 score:",f1)
acc=accuracy_score(y_test,y_pred)
print("accuracy :",acc)

confusion matrix:
 [[94 13]
 [15 32]]
f1 score: 0.6956521739130436
accuracy : 0.8181818181818182


In [6]:
###My KNN
K=11
def KNN(X_train,y_train,X_test,K,classes):
    y_pred=np.zeros((X_test.shape[0],1))
    for i in range(X_test.shape[0]):
        dist=np.sqrt(np.sum(np.square(X_test[i]-X_train),axis=1))
        idxs=np.argsort(-dist.flatten())[:K]
        count=np.zeros((classes,1))
        for idx in idxs:
            for p in range(classes):
                if y_train.iloc[idx]==p:
                    count[p]+=1
        y_pred[i]=np.argmax(count)
    return y_pred
y_pred=KNN(X_train,y_train,X_train,13,2)
print(np.mean(y_pred==y_test.iloc[0]+0))

0.8827361563517915


In [7]:
def findBestK(KNN,X_train,y_train,X_test,classes):
    acc=np.zeros((100,1))
    for k in range(100):
        y_pred=KNN(X_train,y_train,X_test,k,classes)
        acc[k]=np.mean(y_pred==y_test.iloc[0]+0)
    bestK=np.argmax(acc)
    return bestK   
bestK=findBestK(KNN,X_train,y_train,X_test,classes=2)
print(bestK)

5
